# Lesson 03: Reading Images in Python — Extended

There are multiple libraries for loading images in Python. Each handles things slightly differently.  
Understanding these differences prevents subtle bugs.

## Libraries covered:
- **OpenCV (`cv2`)** — BGR order, uint8, most used in computer vision
- **Pillow (`PIL`)** — RGB order, good for general image tasks
- **Matplotlib (`plt.imread`)** — RGB, float or uint8 depending on format
- **NumPy** — direct array representation

## Image formats:
- **JPEG (.jpg)** — lossy compression, smaller file, no transparency
- **PNG (.png)** — lossless compression, supports transparency (alpha channel)
- **BMP (.bmp)** — uncompressed, large files

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

IMAGE_FILE = 'sealion_hero.png'

## 1. Loading with OpenCV vs PIL vs Matplotlib

In [ ]:
# Method 1: OpenCV — loads as BGR uint8
img_cv  = cv2.imread(IMAGE_FILE)
img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)  # must convert for display

# Method 2: PIL — loads as RGB uint8
img_pil = np.array(Image.open(IMAGE_FILE).convert('RGB'))

# Method 3: Matplotlib — loads as RGB (uint8 for PNG, float32 for some formats)
img_mpl = plt.imread(IMAGE_FILE)

print('OpenCV shape / dtype:', img_cv.shape, img_cv.dtype)
print('PIL    shape / dtype:', img_pil.shape, img_pil.dtype)
print('MPL    shape / dtype:', img_mpl.shape, img_mpl.dtype)

# Pixel at (0,0) — check channel order
print('\nPixel (0,0) BGR [OpenCV]:', img_cv[0, 0])
print('Pixel (0,0) RGB [PIL]:   ', img_pil[0, 0])

fig, axes = plt.subplots(1, 3, figsize=(13, 4))
axes[0].imshow(img_rgb); axes[0].set_title('OpenCV (BGR→RGB)'); axes[0].axis('off')
axes[1].imshow(img_pil); axes[1].set_title('PIL (RGB)'); axes[1].axis('off')
axes[2].imshow(img_mpl); axes[2].set_title('Matplotlib'); axes[2].axis('off')
plt.tight_layout()
plt.show()

## 2. Reading Modes — Color, Grayscale, Unchanged

OpenCV `cv2.imread()` has flags to control how the image is loaded:
- `cv2.IMREAD_COLOR` (default) — 3-channel BGR
- `cv2.IMREAD_GRAYSCALE` — 1-channel gray
- `cv2.IMREAD_UNCHANGED` — preserves all channels, including alpha (4th channel if PNG)

In [ ]:
img_color     = cv2.imread(IMAGE_FILE, cv2.IMREAD_COLOR)
img_gray      = cv2.imread(IMAGE_FILE, cv2.IMREAD_GRAYSCALE)
img_unchanged = cv2.imread(IMAGE_FILE, cv2.IMREAD_UNCHANGED)

print('IMREAD_COLOR     shape:', img_color.shape)       # (H, W, 3)
print('IMREAD_GRAYSCALE shape:', img_gray.shape)        # (H, W)
print('IMREAD_UNCHANGED shape:', img_unchanged.shape)   # (H, W, 3 or 4)

fig, axes = plt.subplots(1, 3, figsize=(13, 4))
axes[0].imshow(cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB)); axes[0].set_title('Color (3ch)'); axes[0].axis('off')
axes[1].imshow(img_gray, cmap='gray');                      axes[1].set_title('Grayscale (1ch)'); axes[1].axis('off')
axes[2].imshow(cv2.cvtColor(img_unchanged, cv2.COLOR_BGR2RGB) if img_unchanged.ndim == 3 else img_unchanged, cmap='gray');
axes[2].set_title(f'Unchanged ({img_unchanged.shape[-1] if img_unchanged.ndim==3 else 1}ch)')
axes[2].axis('off')
plt.tight_layout()
plt.show()

## 3. Understanding Image Memory

A 1920×1080 RGB image takes: 1920 × 1080 × 3 bytes = **~6 MB** in memory.

In [ ]:
h, w, c = img_color.shape
size_bytes = img_color.nbytes

print(f'Image dimensions: {w} × {h} pixels')
print(f'Channels: {c}')
print(f'Bits per pixel: {img_color.dtype.itemsize * 8}')
print(f'Memory size: {size_bytes:,} bytes = {size_bytes/1024:.1f} KB = {size_bytes/1024**2:.2f} MB')
print(f'Calculation: {h} × {w} × {c} = {h*w*c:,} bytes')

## 4. Saving in Different Formats and Quality

JPEG uses lossy compression — lower quality = smaller file but more artifacts.

In [ ]:
import os

# Save as PNG (lossless)
cv2.imwrite('output_lossless.png', img_color)

# Save as JPEG with different quality levels
for q in [10, 50, 95]:
    cv2.imwrite(f'output_q{q}.jpg', img_color, [cv2.IMWRITE_JPEG_QUALITY, q])

# Compare file sizes
files = ['output_lossless.png', 'output_q10.jpg', 'output_q50.jpg', 'output_q95.jpg']
for f in files:
    if os.path.exists(f):
        size_kb = os.path.getsize(f) / 1024
        print(f'{f:25s}: {size_kb:8.1f} KB')

In [ ]:
# Visual comparison of JPEG quality
imgs_saved = []
titles = []
for q in [10, 50, 95]:
    fname = f'output_q{q}.jpg'
    if os.path.exists(fname):
        loaded = cv2.imread(fname)
        imgs_saved.append(cv2.cvtColor(loaded, cv2.COLOR_BGR2RGB))
        titles.append(f'JPEG quality={q}')

fig, axes = plt.subplots(1, len(imgs_saved) + 1, figsize=(16, 4))
axes[0].imshow(img_rgb); axes[0].set_title('Original PNG'); axes[0].axis('off')
for ax, im, t in zip(axes[1:], imgs_saved, titles):
    ax.imshow(im); ax.set_title(t); ax.axis('off')
plt.tight_layout()
plt.show()

## 5. Working With Multiple Images

In [ ]:
import glob

# Find all image files in current directory
image_files = glob.glob('*.png') + glob.glob('*.jpg')
print(f'Found {len(image_files)} image(s):')
for f in image_files:
    im = cv2.imread(f)
    if im is not None:
        print(f'  {f:30s} shape={im.shape} dtype={im.dtype}')

## Summary

| Library | Channel Order | Default dtype | Notes |
|---------|--------------|---------------|-------|
| OpenCV | **BGR** | uint8 | Always convert to RGB for display |
| PIL | **RGB** | uint8 | `.convert('RGB')` ensures 3 channels |
| Matplotlib | **RGB** | uint8 or float32 | Depends on file format |

**Key rules:**
- OpenCV → always `cv2.cvtColor(img, cv2.COLOR_BGR2RGB)` before `plt.imshow()`
- JPEG = smaller files but lossy; PNG = lossless with transparency support
- Use `IMREAD_UNCHANGED` to preserve alpha channels in PNG files